<a href="https://colab.research.google.com/github/Illumaria/iti-funds-nav-parser/blob/master/iti_ruse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [0]:
url = 'https://www.londonstockexchange.com/stock/RUSE/iti-funds-ucits-etf-sicav/analysis?page=' # initial link (unspecified page)

In [0]:
resp = requests.get(url + '1')

In [0]:
bs = BeautifulSoup(resp.text)

In [0]:
# Get the number of pages:
pages = bs.find('a', class_="page-number page-last").get('href')
num_pages = int(pages.split(sep='=')[1])

In [0]:
# Find all the partial links on the first page:
scripts_rusb = []
scripts_ruse = []
for tag in bs.find_all('a'):
    if 'EUROBOND: Net Asset Value' in tag.text:
        scripts_rusb.append(tag.get('href'))
    if 'ETF SICAV: Net Asset Value' in tag.text:
        scripts_ruse.append(tag.get('href'))

In [0]:
# Do the same for other pages:
for i in range(2,num_pages+1):
  resp = requests.get('{}{}'.format(url, i))
  bs = BeautifulSoup(resp.text)
  for tag in bs.find_all('a'):
    if 'EUROBOND: Net Asset Value' in tag.text:
      scripts_rusb.append(tag.get('href'))
    if 'ETF SICAV: Net Asset Value' in tag.text:
      scripts_ruse.append(tag.get('href'))
 # urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [0]:
# Get full links:
urls_ruse = []
urls_rusb = []
urls_ruse.extend(['https://www.londonstockexchange.com/' + scripts_ruse[i] for i in range(len(scripts_ruse))])
urls_rusb.extend(['https://www.londonstockexchange.com/' + scripts_rusb[i] for i in range(len(scripts_rusb))])

In [0]:
# Get the number of urls to determine the number of table rows:
nrows_ruse = len(urls_ruse)
nrows_rusb = len(urls_rusb)
# Create an empty pandas dataframe with 0 to (nrows-1) rows and 0 to (2-1) columns:
table_ruse = pd.DataFrame(columns=range(0,4), index = range(0, nrows_ruse))
table_rusb = pd.DataFrame(columns=range(0,4), index = range(0, nrows_rusb))

In [41]:
#resp = requests.get(urls_ruse[1])
#bs = BeautifulSoup(resp.text)
#date = bs.find('script', text = re.compile('DEALING DATE'))
#date_pattern = re.compile(r'DEALING DATE: \d+\-\w{2,3}\-\d{4}')

#match = re.search(date_pattern, str(date))
#print(match.group(0).split()[2])

#for match in re.findall(pattern, str(date)):
#    print(match)
# DEALING DATE: 09-Jun-2020

{&q;G.{{api_endpoint}}/api/v1/pages?parameters=newsId%3D14572680&a;path=news-article&q;:{&q;body&q;:{&q;id&q;:&q;8&q;,&q;layout&q;:&q;onecol&q;,&q;info&q;:{&q;title&q;:&q;News article&q;,&q;headers&q;:{&q;description&q;:&q;News article - Regulatory News Service&q;,&q;keywords&q;:&q;lseg stock prices&q;,&q;robots&q;:null}},&q;meta&q;:{&q;title&q;:&q;News article&q;,&q;page_type&q;:&q;news-article&q;,&q;page_background&q;:null,&q;available_languages&q;:[&q;en&q;],&q;language&q;:&q;en&q;},&q;components&q;:[{&q;id&q;:&q;footer&q;,&q;timestamp&q;:null,&q;meta&q;:{&q;col&q;:0,&q;title&q;:&q;footer&q;},&q;position&q;:&q;footer&q;,&q;type&q;:&q;footer&q;,&q;weight&q;:0,&q;staticContent&q;:null,&q;content&q;:{&q;footerlinks&q;:{&q;label&q;:&q;-&q;,&q;value&q;:[{&q;title&q;:&q;Disclaimer&q;,&q;uri&q;:&q;/disclaimer&q;,&q;target&q;:&q;_self&q;},{&q;title&q;:&q;Privacy and cookie policy&q;,&q;uri&q;:&q;/privacy-and-cookie-policy&q;,&q;target&q;:&q;_self&q;},{&q;title&q;:&q;Conflict of interest pol

In [44]:
# Fill the dataframe with data:
for row in range(nrows_ruse):
  # This is going to take some time... Uncomment to see the "progress bar" :D
  #print(url)
  resp = requests.get(urls_ruse[row])
  bs = BeautifulSoup(resp.text)
  script = bs.find('script', text = re.compile('DEALING DATE')).text

  date = re.search(re.compile(r'DEALING DATE: \d+\-\w{2,3}\-\d{4}'), script).group(0)
  table_ruse.iat[row,0] = date.split()[2]

  #nav = bs.find('p', text = re.compile('NAV PER SHARE')).text
  nav = re.search(re.compile(r'NAV PER SHARE: USD: \d+.\d+'), script).group(0)
  table_ruse.iat[row,1] = float(nav.split()[4])

  #shares = bs.find('p', text = re.compile('NUMBER OF SHARES')).text
  shares = re.search(re.compile(r'NUMBER OF SHARES IN ISSUE: \d+'), script).group(0)
  table_ruse.iat[row,2] = int(shares.split()[5])

  table_ruse.iat[row,3] = fnav*ishares

table_ruse

NameError: ignored

In [0]:
# Rename the columns:
table.columns = ['Date', 'NAV per share', 'Number of shares', 'Total NAV']
table

In [0]:
# Set the date column to datetime type:
table['Date'] = pandas.to_datetime(table['Date'], dayfirst=True)
table

In [0]:
# Do some type conversions:
table['NAV per share'] = pandas.to_numeric(table['NAV per share'])
table['Number of shares'] = pandas.to_numeric(table['Number of shares'])
table['Total NAV'] = pandas.to_numeric(table['Total NAV'])
# Change the index column:
table.index = pandas.DatetimeIndex(table['Date'])
table

In [0]:
# Get rid of the data column since it copies the index column now:
table = table.drop(columns=['Date']).dropna()
table

In [0]:
# Inverse the dataframe:
table = table.iloc[::-1]
table